In [1]:
##################
# Libraries
##################
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC, SVC
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from loguru import logger

In [2]:
##################
# Constants
##################
MODEL_SEED=42
ROWS_SEED=[24, 42, 206, 602, 412, 214, 754, 457, 2023, 3202]
SIZE_OF_UNLABELLED_DATA = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9]
MODELS = [
    GaussianNB(var_smoothing=0.01519911082952933),
    DecisionTreeClassifier(criterion='entropy', max_depth=9, min_samples_leaf=3, min_samples_split=3),
    KNeighborsClassifier(metric='manhattan', n_neighbors=15, weights='distance'),
    RandomForestClassifier(criterion='gini', max_depth=15, min_samples_leaf=1, min_samples_split=9, n_estimators=400),
]

In [3]:
##################
# Functions
##################
def encode_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Encode dataframe using LabelEncoder"""
    labelencoder = LabelEncoder()

    for col in df.columns:
        df[col] = labelencoder.fit_transform(df[col])

    return df

def get_metrics() -> dict:
    metrics = {
        'accuracy': accuracy_score,
        'precision': precision_score,
        'recall': recall_score,
        'f1': f1_score,
        'roc': roc_auc_score
    }
    return metrics


def get_metrics_df(y_true, y_pred) -> pd.DataFrame:
    metrics = get_metrics()
    df = pd.DataFrame()
    for metric_name, metric in metrics.items():
        df[metric_name] = [metric(y_true, y_pred)]

    return df

def run_pipeline(X_train, X_test, y_train, y_test, size_of_unlabelled_data=0.5, model=None, rows_seed=42) -> dict:
    if model is None:
        raise ValueError("Model is None")
    
    X_train = X_train.copy()
    X_test = X_test.copy()
    y_train = y_train.copy()
    y_test = y_test.copy()

    # Unlabelled data
    rng = np.random.RandomState(rows_seed)
    random_rows_with_rng = rng.choice(X_train.index, size=int(len(X_train)*size_of_unlabelled_data), replace=False)
    y_train.loc[random_rows_with_rng] = -1

    # Scale data
    scaler = RobustScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)


    self_training_model = SelfTrainingClassifier(model, verbose=False)
    self_training_model.fit(X_train, y_train)

    # Predict
    y_pred = self_training_model.predict(X_test)

    # Evaluate - Get Metrics
    metrics = get_metrics_df(y_test, y_pred)
    accuracy = metrics["accuracy"].values[0]
    precision = metrics["precision"].values[0]
    recall = metrics["recall"].values[0]
    f1 = metrics["f1"].values[0]
    roc = metrics["roc"].values[0]
    

    return {"rows_seed": rows_seed, "model": type(model).__name__, "unlabeled": size_of_unlabelled_data, "acc": accuracy, "precision": precision, "recall": recall, "f1": f1, "roc": roc}

def run_pipeline_self(X_train, X_test, y_train, y_test) -> pd.DataFrame:
    run_counter = 0
    results = []

    for model in MODELS:
        for value in SIZE_OF_UNLABELLED_DATA:
            logger.info(f'Model: {model}')
            logger.info(f'Size of unlabelled data: {value}')
            for row_seed in ROWS_SEED:
                run_counter += 1
                logger.info(f"%{run_counter} - Running pipeline for row_seed: {row_seed} and size_of_unlabelled_data: {value} and model: {model}")
                result = run_pipeline(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, size_of_unlabelled_data=value, model=model, rows_seed=row_seed)
                results.append(result)
            print('\n')

    results_rows_seed = []
    results_models = []
    results_unlabeled = []
    results_acc = []
    results_precision = []
    results_recall = []
    results_f1 = []
    results_roc = []

    for result in results:
        results_rows_seed.append(result.get('rows_seed'))
        results_models.append(result.get('model'))
        results_unlabeled.append(result.get('unlabeled'))
        results_acc.append(result.get('acc'))
        results_precision.append(result.get('precision'))
        results_recall.append(result.get('recall'))
        results_f1.append(result.get('f1'))
        results_roc.append(result.get('roc'))


    results_df = pd.DataFrame({'rows_seed': results_rows_seed, 'model': results_models, 'unlabeled': results_unlabeled, 'acc': results_acc, 'precision': results_precision, 'recall': results_recall, 'f1': results_f1, 'roc': results_roc})

    return results_df

    

In [6]:
df = pd.read_csv('../../datasets/water_potability.csv')
df = df.dropna()

X = df.drop('Potability', axis=1)
y = df['Potability']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=MODEL_SEED)

In [7]:
results_df = run_pipeline_self(X_train, X_test, y_train, y_test)

2023-10-22 14:34:21.235 | INFO     | __main__:run_pipeline_self:75 - Model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:21.235 | INFO     | __main__:run_pipeline_self:76 - Size of unlabelled data: 0.1
2023-10-22 14:34:21.236 | INFO     | __main__:run_pipeline_self:79 - %1 - Running pipeline for row_seed: 24 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:21.252 | INFO     | __main__:run_pipeline_self:79 - %2 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:21.266 | INFO     | __main__:run_pipeline_self:79 - %3 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:21.279 | INFO     | __main__:run_pipeline_self:79 - %4 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.01519

2023-10-22 14:34:21.603 | INFO     | __main__:run_pipeline_self:79 - %23 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:21.622 | INFO     | __main__:run_pipeline_self:79 - %24 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:21.643 | INFO     | __main__:run_pipeline_self:79 - %25 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:21.661 | INFO     | __main__:run_pipeline_self:79 - %26 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:21.678 | INFO     | __main__:run_pipeline_self:79 - %27 - Running pipeline for row_seed: 754 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.01519911082952933)


2023-10-22 14:34:21.961 | INFO     | __main__:run_pipeline_self:75 - Model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:21.962 | INFO     | __main__:run_pipeline_self:76 - Size of unlabelled data: 0.5
2023-10-22 14:34:21.962 | INFO     | __main__:run_pipeline_self:79 - %41 - Running pipeline for row_seed: 24 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:21.996 | INFO     | __main__:run_pipeline_self:79 - %42 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:22.020 | INFO     | __main__:run_pipeline_self:79 - %43 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:22.044 | INFO     | __main__:run_pipeline_self:79 - %44 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.0

2023-10-22 14:34:22.170 | INFO     | __main__:run_pipeline_self:79 - %50 - Running pipeline for row_seed: 3202 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:22.188 | INFO     | __main__:run_pipeline_self:75 - Model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:22.188 | INFO     | __main__:run_pipeline_self:76 - Size of unlabelled data: 0.6
2023-10-22 14:34:22.189 | INFO     | __main__:run_pipeline_self:79 - %51 - Running pipeline for row_seed: 24 and size_of_unlabelled_data: 0.6 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:22.209 | INFO     | __main__:run_pipeline_self:79 - %52 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.6 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:22.229 | INFO     | __main__:run_pipeline_self:79 - %53 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.6 and model: GaussianNB(var_smoothing=0.

2023-10-22 14:34:22.394 | INFO     | __main__:run_pipeline_self:79 - %62 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.7 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:22.412 | INFO     | __main__:run_pipeline_self:79 - %63 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.7 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:22.429 | INFO     | __main__:run_pipeline_self:79 - %64 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.7 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:22.447 | INFO     | __main__:run_pipeline_self:79 - %65 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.7 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:34:22.463 | INFO     | __main__:run_pipeline_self:79 - %66 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.7 and model: GaussianNB(var_smoothing=0.01519911082952933)
2

...

2023-10-22 14:43:03.975 | INFO     | __main__:run_pipeline_self:79 - %352 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:43:13.810 | INFO     | __main__:run_pipeline_self:79 - %353 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:43:22.634 | INFO     | __main__:run_pipeline_self:79 - %354 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:43:32.451 | INFO     | __main__:run_pipeline_self:79 - %355 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:43:43.655 | INFO     | __main__:run_pipeline_self:79 - %356 - R

2023-10-22 14:44:39.325 | INFO     | __main__:run_pipeline_self:79 - %362 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:44:49.913 | INFO     | __main__:run_pipeline_self:79 - %363 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:45:00.090 | INFO     | __main__:run_pipeline_self:79 - %364 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:45:09.281 | INFO     | __main__:run_pipeline_self:79 - %365 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:45:21.083 | INFO     | __main__:run_pipeline_self:79 - %366 - R

2023-10-22 14:46:21.705 | INFO     | __main__:run_pipeline_self:79 - %372 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:46:32.876 | INFO     | __main__:run_pipeline_self:79 - %373 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:46:43.064 | INFO     | __main__:run_pipeline_self:79 - %374 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:46:53.008 | INFO     | __main__:run_pipeline_self:79 - %375 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:47:05.435 | INFO     | __main__:run_pipeline_self:79 - %376 - R

2023-10-22 14:48:05.864 | INFO     | __main__:run_pipeline_self:79 - %382 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:48:16.390 | INFO     | __main__:run_pipeline_self:79 - %383 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:48:28.563 | INFO     | __main__:run_pipeline_self:79 - %384 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:48:38.138 | INFO     | __main__:run_pipeline_self:79 - %385 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:48:49.183 | INFO     | __main__:run_pipeline_self:79 - %386

2023-10-22 14:49:53.671 | INFO     | __main__:run_pipeline_self:79 - %392 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:50:04.030 | INFO     | __main__:run_pipeline_self:79 - %393 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:50:13.293 | INFO     | __main__:run_pipeline_self:79 - %394 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:50:22.300 | INFO     | __main__:run_pipeline_self:79 - %395 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=15, min_samples_split=9, n_estimators=400)
2023-10-22 14:50:31.531 | INFO     | __main__:run_pipeline_self:79 - %396 - R

In [8]:
# mean and standard deviation for all metrics
results_df_with_mean_std = results_df.groupby(['model', 'unlabeled']).agg({'acc': ['mean', 'std'], 'f1': ['mean', 'std'], 'precision': ['mean', 'std'], 'recall': ['mean', 'std'], 'roc': ['mean', 'std']})
results_df_with_mean_std = results_df_with_mean_std.reset_index(level=1)
results_df_with_mean_std.columns = ["_".join(col) for col in results_df_with_mean_std.columns.values]
results_df_with_mean_std.rename(columns={'unlabeled_': 'unlabeled'}, inplace=True)
results_df_with_mean_std.reset_index(inplace=True)
results_df_with_mean_std.columns

Index(['model', 'unlabeled', 'acc_mean', 'acc_std', 'f1_mean', 'f1_std',
       'precision_mean', 'precision_std', 'recall_mean', 'recall_std',
       'roc_mean', 'roc_std'],
      dtype='object')

In [9]:
# Metrics with tabulate
from tabulate import tabulate

print(tabulate(results_df_with_mean_std, headers='keys', tablefmt='psql', showindex=False))

+------------------------+-------------+------------+-------------+------------+------------+------------------+-----------------+---------------+--------------+------------+-------------+
| model                  |   unlabeled |   acc_mean |     acc_std |    f1_mean |     f1_std |   precision_mean |   precision_std |   recall_mean |   recall_std |   roc_mean |     roc_std |
|------------------------+-------------+------------+-------------+------------+------------+------------------+-----------------+---------------+--------------+------------+-------------|
| DecisionTreeClassifier |        0.1  |   0.623573 | 0.0190078   | 0.473598   | 0.0603724  |         0.592388 |       0.0432474 |   0.404651    |   0.0858507  |   0.595616 | 0.020058    |
| DecisionTreeClassifier |        0.2  |   0.616873 | 0.0183541   | 0.465505   | 0.0421481  |         0.579696 |       0.0374585 |   0.395349    |   0.0679125  |   0.588584 | 0.016758    |
| DecisionTreeClassifier |        0.3  |   0.613151 | 0